
# Database & SQL for Data Science Task

Agus Wibowo - 950220

Diketahui dalam sebuah minimarket  deltamart, memiliki beberapa data sebagai berikut :
* CUSTOMER, merupakan data profil customer minimarket 
* ORDER_TRANSACTION, riwayat pembelanjaan  customer
* ORDERITEM, detil pembelanjaan customer
* PRODUCT, Profil product yang dimiliki minimarket


Perusahaan tersebut ingin melakukan analisa pemodelan, dengan memprediksi apakah customer akan melakukan pembayaran secara cash atau tidak. Maka anda sebagai data scientist sebelum melakukan pemodelan akan mempersiapkan datamart terlebih dahulu.

| COLUMN       | DESCRIPTION           | EXAMPLE  |
| :-------------|:-------------| :-----|
| ID    | Customer ID | 1 |
| Name      | Gabungan FirstName & LastName | Anton Suhartono |
| Country | Asal Negara      |    Germany |
| Flag_phone    | Jika Ada Nomor Telfonnya, Tulis 1, Jika Tidak Maka 0 | 1 |
| Age      | Umur, Kalau Tidak Ada Datanya Diasumsikan 25 | 17 |
| LOS | Lama Berlangganan Dihitung Semenjak Awal Order Barang, Dalam Satuan Hari      |    365 |
| TotalProduct    | Total product seluruh yang pernah dibeli | 50 |
| TotalAmount    | Total uang yang dikeluarkan dari seluruh barang yang dibeli | 5000000|
| PaymentType | Type pembayaran, jika masih dalam bentuk cash maka ditulis 1, lainnya 0      |    1 |

Perlu diperhatikan bahwasannya hasilnya berupa table berukuran 9 kolom 91 baris.

# Install Packages

In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql

In [2]:
%load_ext sql

## Connect to the Database


In [3]:
%sql ibm_db_sa://khs44136:VHHMKIKAR7RQqixK@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/BLUDB?security=SSL

## Query Data

In [4]:
%%sql
SELECT 
	C.ID, 
    COALESCE(C.Firstname ||' '||C.Lastname, C.Firstname) AS NAME,
    C.COUNTRY,
    CASE
    	WHEN C.Phone is NULL THEN 0
        ELSE 1
    END AS Flag_Phone,
    COALESCE(C.AGE,25) AS AGE,
    CASE
    	WHEN OT.OrderDate is NULL THEN 0
        ELSE (DAYS('2021-07-23') - DAYS(OT.OrderDate))
    END AS LOS,
    CASE
    	WHEN OI.TOTAL_QUANTITY is NULL THEN 0
        ELSE OI.TOTAL_QUANTITY
    END AS TotalProduct,
    CASE
    	WHEN OT.TotalAmount is NULL THEN 0
        ELSE OT.TotalAmount
    END AS TotalAmount,
    CASE
    	WHEN OI.PAYMENT is NULL THEN 1
        ELSE OI.PAYMENT
    END AS PaymentType
FROM Customer C 
LEFT JOIN 
(SELECT CustomerID, MIN(OrderDate) AS OrderDate, SUM(TotalAmount) AS TotalAmount FROM Order_transaction GROUP BY CustomerID) AS OT
ON C.ID=OT.CustomerID
LEFT JOIN
(SELECT 
 	CUSTOMERID, 
    SUM(QUANTITY) AS TOTAL_QUANTITY,
    MIN(CASE
     	WHEN PaymentType = 'Cash' THEN 1
        ELSE 0
    END) AS PAYMENT
FROM (SELECT OrderId, SUM(Quantity) AS QUANTITY FROM OrderItem GROUP BY OrderID) AS OrderItem2 INNER JOIN Order_Transaction
ON Order_Transaction.ID=OrderItem2.OrderId GROUP BY Order_Transaction.CUSTOMERID) AS OI
ON C.ID=OI.CustomerID
/*ORDER BY C.ID*/;

 * ibm_db_sa://khs44136:***@ba99a9e6-d59e-4883-8fc0-d6a8c9f7a08f.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:31321/BLUDB
Done.


id,name,country,flag_phone,age,los,totalproduct,totalamount,paymenttype
1,Maria Anders,Germany,1,31,2889,174,4596.20,0
2,UvuvwevwevweOnyetenvewveUgwemubwemOssas Trujillo,Mexico,1,26,3230,63,1402.95,0
3,Antonio Moreno,Mexico,1,28,3160,359,7515.35,0
4,Thomas Hardy,UK,1,30,3172,650,13806.50,0
5,Christina Berglund,Sweden,1,25,3267,1001,26968.15,0
6,Hanna Moos,Germany,1,35,3027,140,3239.80,0
7,Frédérique Citeaux,France,1,31,3285,666,19088.00,0
8,Martín Sommer,Spain,1,28,3208,190,5297.80,0
9,Laurence Lebihan,France,0,18,3202,980,23850.95,0
10,Elizabeth Lincoln,Canada,1,26,3137,956,22607.70,0
